In [2]:
import pandas as pd
import nltk
import re

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


In [5]:
stemmer = PorterStemmer()
lemmatize = WordNetLemmatizer()


In [ ]:
paragraph = """
Chief Bola Ahmed Adekunle Tinubu GCFR (born 29 March 1952) is a Nigerian politician who is the 16th and current president of Nigeria.[1] He was the governor of Lagos State from 1999 to 2007; and senator for Lagos West in the Third Republic.

Tinubu spent his early life in southwestern Nigeria and later moved to the United States where he studied accounting at Chicago State University. He returned to Nigeria in the early 1990s and was employed by Mobil Nigeria as an accountant, before entering politics as a Lagos West senatorial candidate in 1992 under the banner of the Social Democratic Party. After dictator Sani Abacha dissolved the Senate in 1993, Tinubu became an activist campaigning for the return of democracy as a part of the National Democratic Coalition movement.

In the first post-transition Lagos State gubernatorial election, Tinubu won by a wide margin as a member of the Alliance for Democracy. Four years later, he won re-election to a second term. After leaving office in 2007, he played a key role in the formation of the All Progressives Congress in 2013. In 2023, he was elected president of Nigeria.
"""

In [7]:
# tokenization
nltk.download('stopwords')
nltk.download('punkt')

# sentences = nltk.sent_tokenize(paragraph)

[nltk_data] Downloading package stopwords to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
corpus = []
for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review=review.lower()
    corpus.append(review)

In [ ]:
# to apply stemming
for i in corpus:
    words = nltk.word_tokenize(i)
    for word in words:
        if word not in set(stopwords.words('english')):
            print(stemmer.stem(word))

In [ ]:
stopwords.words('english')
nltk.download('wordnet')


In [ ]:
# paragraph = []

# for i in range(len(sentences)):
#     review = re.sub('[^a-zA-Z]', ' ', sentences[i])
#     review = review.lower()
#     review = review.split()
#     review = [lemmatize.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
#     review = ' '.join(review) 
#     paragraph.append(review)

In [ ]:
# to apply lemmatizer

for i in corpus:
    words = nltk.word_tokenize(i)
    for word in words:
        if word not in set(stopwords.words('english')):
            print(lemmatize.lemmatize(word))

In [ ]:
# bag of words

cv = CountVectorizer(binary=True, ngram_range=(3,3))

In [ ]:
X = cv.fit_transform(corpus)
cv.vocabulary_
sorted(cv.vocabulary_.items())


In [ ]:
X[0].toarray()

In [ ]:
corpus

In [ ]:
## TFIDF
# from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(ngram_range=(1,1), max_features=10)
X = tv.fit_transform(corpus)

In [ ]:
X[0].toarray()

In [164]:
# READ CSV
spam = pd.read_csv('spam_ham.csv', sep='\t', names=['labels', 'message'])

In [ ]:
# APPLY STEMMER
corpus_stem = []

for i in range(0, len(spam)):
    review = re.sub('[^a-zA-Z0-9]', ' ', spam['message'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) 
    corpus_stem.append(review)

In [ ]:
# bag of words
cv = CountVectorizer(max_features=2500,binary=True)
X = cv.fit_transform(corpus_stem).toarray()

In [89]:
y = pd.get_dummies(spam['labels'])
y = y.iloc[:, 1].astype(int)

In [91]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: spam, Length: 5572, dtype: int32

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
mnb = MultinomialNB().fit(X_train, y_train)
# prediction
y_pred = mnb.predict(X_test)
accuracy_score(y_pred, y_test)

In [ ]:
# classification_report(y_pred, y_test)
confusion_matrix(y_pred, y_test)

In [ ]:
# TFIDF

tv = TfidfVectorizer(max_features=2500, ngram_range=(1,2))
X = tv.fit_transform(corpus_stem).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
mnb = MultinomialNB().fit(X_train, y_train)
y_pred = mnb.predict(X_test)
accuracy_score(y_pred, y_test)

In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_pred, y_test)


### Word2vec

In [ ]:
%pip install gensim

In [21]:
import numpy as np

In [ ]:
# import gensim.downloader as api

# wv = api.load('word2vec-google-news-300')

In [165]:
corpus = []

for i in range(0, len(spam)):
    review = re.sub('[^a-zA-Z0-9]', ' ', spam['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatize.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) 
    corpus.append(review)

In [167]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [ ]:
y

In [109]:
y = pd.get_dummies(y).iloc[:, 1].astype(int)

In [110]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [174]:
# words = []
# for i in corpus:
#     sent_token = sent_tokenize(i)
#     for i in sent_token:
#         words.append(simple_preprocess(i))

In [195]:
words = []
for i in corpus:
    sent_token = sent_tokenize(i)
    words.append(simple_preprocess(i))
        

In [197]:
len(words)

5565

In [198]:
len(words)

5565

In [154]:
model = gensim.models.Word2Vec(words, window=5,min_count=2)

In [157]:
model.wv.index_to_key

315

In [158]:
model.corpus_count

5572

In [142]:
model.epochs

5

In [160]:
model.wv.similar_by_word('prize')

KeyError: "Key 'prize' not present in vocabulary"

In [144]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    # sent = [word for word in doc if word in model.mv.index_to_key]
    # print(sent)
    
    return np.mean([model.wv[word] for word in words if word in model.wv.index_to_key], axis=0)


In [27]:
from tqdm import tqdm

In [162]:
X = []

for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5572 [00:00<?, ?it/s]c:\Users\ACER-PC\Desktop\Z\BA_nltk\.venv\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\ACER-PC\Desktop\Z\BA_nltk\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 5572/5572 [07:49<00:00, 11.87it/s]


In [163]:
X

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [41]:
X_new =  np.array(X, dtype=object)

In [45]:
X_new[0]

array([-0.0572608 ,  0.2736493 ,  0.02597114,  0.01283087,  0.01862401,
       -0.38753533,  0.06568547,  0.51432174, -0.14124586, -0.12946813,
       -0.10796515, -0.35931024, -0.0528903 ,  0.08321051,  0.07744681,
       -0.24833035,  0.00578281, -0.3561922 , -0.01361859, -0.40739238,
        0.15922056,  0.04068476,  0.11397326, -0.09942146, -0.11764296,
        0.03255244, -0.24111311, -0.2693658 , -0.241899  ,  0.07553742,
        0.31912678, -0.00396283,  0.11117529, -0.17404239, -0.09871449,
        0.2199322 , -0.06809594, -0.23408274, -0.14690335, -0.49812046,
       -0.00717556, -0.27797118, -0.05698733,  0.03035105,  0.21321188,
       -0.18032382, -0.17816865,  0.0175348 ,  0.10231765,  0.1819616 ,
        0.12994821, -0.20816559, -0.03378843, -0.04842072, -0.18076712,
        0.11473767,  0.13337092,  0.03621004, -0.24753475, -0.02266791,
        0.02906995,  0.13778256, -0.11427843,  0.04078652, -0.36477742,
        0.32023242,  0.11158089,  0.21156524, -0.33866885,  0.35

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=0)

ValueError: Found input variables with inconsistent numbers of samples: [5565, 5572]

In [76]:
len(y), len(X)

(5572, 5565)

In [79]:
y

0       False
1       False
2        True
3       False
4       False
        ...  
5567     True
5568    False
5569    False
5570    False
5571    False
Name: spam, Length: 5572, dtype: bool